In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.2. Biomarker Distribution Analysis/Output')
cur_date = "050322"

library(readxl)
library(tidyverse)
library(reshape2)

# reading in files
proteomics_5_df = data.frame(read_excel("Input/Proteomics_Data_5ug_040423.xlsx", sheet = 2))
proteomics_25_df = data.frame(read_excel("Input/Proteomics_Data_25ug_040423.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
head(proteomics_25_df)

,Subject_No,Subject_ID,Protein_Accession,Gene_Name,Description,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,1488600000,30.47131
2,2,M_2,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3806600000,31.82586
3,3,M_3,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3900000000,31.86083
4,4,F_4,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5373200000,32.32313
5,5,F_5,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,2595200000,31.27320
6,6,M_6,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5780700000,32.42860


Testing for statistical differences by comparing a protein expression between burn conditions (control, smoldering or flaming). (The condensate and concentration would remain consistent). Protein expression (intensity) was pseudo log transformed to reduce skew. 

Typically, normality and homogeneity of variances would be tested for first, however a non-parametric test (Wilcoxon rank sum) will be used given the small sample size (*n* = 6). 

In [3]:
wilcoxon_rank_sumv1 = function(df){
    # """
    # Running wilcoxon rank sums after filtering for a protein and condensate using a loop. 
    # Ultimately using this test to compare protein expression (control vs. flaming burn condition).

    # :param: protein dataframe
    # :output: a dataframe containing the protein, condensate, comparison, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    proteins = unique(df$Protein_Accession)
    condensates = c("C", "P")
    
    values_df = data.frame()
     # iterating through each protein, condensate, and burn condition
    for(i in 1:length(proteins)){
        for(j in 1:length(condensates)){
                
            # control df
            control_df = df %>%
                filter(Protein_Accession == proteins[i], Condensate == "PBS")

            # flaming df
            flaming_df = df %>%
                filter(Protein_Accession == proteins[i], Condensate == condensates[j], Burn_Condition == "F")

            # wilcoxon rank sum
            wilcoxon_test = wilcox.test(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, paired = TRUE)

            # contains gene, condensate, burn conditions, conc, stat, and p value
            values_vector = cbind(proteins[i], condensates[j], "Control vs. Flaming",
                                  unique(df$Concentration), wilcoxon_test$statistic, wilcoxon_test$p.value)
            values_df = rbind(values_df, values_vector)
        }
    }

    
    # adding col names
    colnames(values_df) = c("Protein", "Condensate", "Comparison", "Concentration", "Statistic", "P Value")
    
    # calculating padj values
    values_df$`P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")

    
    return(values_df)
}

In [4]:
# calling fn
conc25_control_wilcox_values = wilcoxon_rank_sumv1(proteomics_25_df)
conc5_control_wilcox_values = wilcoxon_rank_sumv1(proteomics_5_df)

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

...

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

In [5]:
wilcoxon_rank_sumv2 = function(df){
    # """
    # Running wilcoxon rank sums after filtering for a protein and condensate using a loop. 
    # Ultimately using this test to compare protein expression (smoldering vs. flaming burn condition).

    # :param: protein dataframe
    # :output: a dataframe containing the protein, condensate, comparison, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    proteins = unique(df$Protein_Accession)
    condensates = c("C", "P")
    
    values_df = data.frame()
     # iterating through each protein, condensate, and burn condition
    for(i in 1:length(proteins)){
        for(j in 1:length(condensates)){
                
            # control df
            control_df = df %>%
                filter(Protein_Accession == proteins[i], Condensate == condensates[j], Burn_Condition == "S")

            # flaming df
            flaming_df = df %>%
                filter(Protein_Accession == proteins[i], Condensate == condensates[j], Burn_Condition == "F")

            # wilcoxon rank sum
            wilcoxon_test = wilcox.test(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, paired = TRUE)

            # contains gene, condensate, burn conditions, conc, stat, and p value
            values_vector = cbind(proteins[i], condensates[j], "Smoldering vs. Flaming",
                                  unique(df$Concentration), wilcoxon_test$statistic, wilcoxon_test$p.value)
            values_df = rbind(values_df, values_vector)
        }
    }

    
    # adding col names
    colnames(values_df) = c("Protein", "Condensate", "Comparison", "Concentration", "Statistic", "P Value")
    
    # calculating padj values
    values_df$`P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")

    
    return(values_df)
}

In [6]:
# calling fn
conc25_smoldering_wilcox_values = wilcoxon_rank_sumv2(proteomics_25_df)
conc5_smoldering_wilcox_values = wilcoxon_rank_sumv2(proteomics_5_df)

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

...

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warning message in wilcox.test.default(control_df$Intensity_pslog2, flaming_df$Intensity_pslog2, :
“cannot compute exact p-value with zeroes”
Warnin

In [7]:
# creating 1 df
proteomics_wilcoxon_df = rbind(conc25_control_wilcox_values, conc5_control_wilcox_values, 
                               conc25_smoldering_wilcox_values, conc5_smoldering_wilcox_values) %>%
    # changing condensates so they're more legible
    mutate(Condensate = ifelse(Condensate == "C", "Cardboard",
                               ifelse(Condensate == "P", "Plastic", "PBS")))

head(proteomics_wilcoxon_df)

,Protein,Condensate,Comparison,Concentration,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
V,CO3_HUMAN,Cardboard,Control vs. Flaming,25,10,1,1.0000000
V1,CO3_HUMAN,Plastic,Control vs. Flaming,25,8,0.6875,0.9082963
V2,K1C19_HUMAN,Cardboard,Control vs. Flaming,25,1,0.0625,0.3600251
V3,K1C19_HUMAN,Plastic,Control vs. Flaming,25,1,0.0625,0.3600251
V4,TBB4B_HUMAN,Cardboard,Control vs. Flaming,25,3,0.15625,0.3776904
V5,TBB4B_HUMAN,Plastic,Control vs. Flaming,25,3,0.15625,0.3776904


In [8]:
# exporting
write.csv(proteomics_wilcoxon_df, paste0(Output,"/", cur_date, "_Proteomics_Wilcoxon_Results.csv"), row.names = FALSE)